## Goal:

<insert Kelli's message from slack here>

## Imports

In [19]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import numpy as np

## Data read-in

In [52]:
df = pd.read_csv('CHI_agents_data-Table 1.csv',low_memory=False)

In [54]:
# Function to convert WKT format to a Point geometry
def to_point(coord):
    try:
        if isinstance(coord, str) and 'POINT' in coord:
            # Use WKT to convert the string into a Point object
            return wkt.loads(coord)
        else:
            return None  # Return None for invalid or missing coordinates
    except Exception as e:
        return None  # Catch any issues

# Apply the conversion to the 'geometry' column
df['geometry'] = df['geometry'].apply(to_point)

# Now convert the DataFrame to a GeoDataFrame
df_geo = gpd.GeoDataFrame(df, geometry='geometry')

In [55]:
boundaries_df = gpd.read_file('Boundaries - Community Areas (current).geojson')

In [56]:
communities_to_keep = [
    'NEAR SOUTH SIDE',
    'ARMOUR SQUARE',
    'BRIDGEPORT',
    'MCKINLEY PARK',
    'BRIGHTON PARK',
    'ARCHER HEIGHTS',
    'GARFIELD RIDGE',
    'CLEARING',
    'WEST ELSDON',
    'GAGE PARK',
    'NEW CITY',
    'FULLER PARK',
    'GRAND BOULEVARD',
    'DOUGLAS',
    'OAKLAND',
    'KENWOOD',
    'HYDE PARK',
    'WASHINGTON PARK',
    'ENGLEWOOD',
    'WEST ENGLEWOOD',
    'CHICAGO LAWN',
    'WEST LAWN',
    'WOODLAWN',
    'GREATER GRAND CROSSING',
    'SOUTH SHORE',
    'AUBURN GRESHAM',
    'ASHBURN',
    'CHATHAM',
    'AVALON PARK',
    'SOUTH CHICAGO',
    'CALUMET HEIGHTS',
    'BURNSIDE',
    'WASHINGTON HEIGHTS',
    'BEVERLY',
    'MOUNT GREENWOOD',
    'MORGAN PARK',
    'WEST PULLMAN',
    'SOUTH DEERING',
    'RIVERDALE',
    'EAST SIDE',
    'HEGEWISCH'
]

In [57]:
filtered_boundaries = boundaries_df[boundaries_df['community'].isin(communities_to_keep)]

In [58]:
df_geo = df_geo.set_crs(4326)

In [74]:
# Read in neighborhood boundaries
neighborhood_boundaries = filtered_boundaries

# Clean and convert the 'Price' column to integers
df_geo['Price_int'] = df_geo['Price'].str.replace('[\$,]', '', regex=True).astype(int)

# Ensure both GeoDataFrames (df_geo and neighborhood_boundaries) have the same CRS
if df_geo.crs != neighborhood_boundaries.crs:
    df_geo = df_geo.to_crs(neighborhood_boundaries.crs)  # Transform CRS to match

# Perform a spatial join to find which points (brokerages) fall within which community
joined_gdf = gpd.sjoin(df_geo, neighborhood_boundaries, how="inner", predicate='within')

# # Group by community and agent/team to calculate counts and total sales
# grouped = joined_gdf.groupby(['community', 'Final_Agent/Team']).agg(
#     count=('Final_Agent/Team', 'count'),
#     total_price=('Price_int', 'sum')
# ).reset_index().nlargest(20)

# # Find the agent/team with the highest count for each community
# max_count_idx = grouped.groupby("community")['count'].idxmax()
# agent_result = grouped.loc[max_count_idx, ['community', 'Final_Agent/Team', 'count', 'total_price']]
# agent_result = agent_result.rename(columns={'count': 'agent_count', 'total_price': 'agent_price'})

# # Calculate community-level totals
# community_result = grouped.groupby('community').agg(
#     community_count=('count', 'sum'),
#     community_price=('total_price', 'sum')
# ).reset_index()

# # Merge agent results with community-level totals
# merged_df = pd.merge(agent_result, community_result, on='community', how='inner')

# # Drop unnecessary columns from neighborhood boundaries
# columns_to_drop = ['area', 'shape_area', 'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len']
# neighborhood_boundaries = neighborhood_boundaries.drop(columns=columns_to_drop, errors='ignore')

# # Merge with neighborhood boundaries
# final_merge = neighborhood_boundaries.merge(merged_df, on='community')
# final_merge = final_merge.rename(columns={'Final_Agent/Team': 'final_agent_team'})

# # Save the result to a GeoJSON file
# final_merge.to_file("final_merge.geojson", driver='GeoJSON')


In [82]:
joined_gdf.value_counts('Final_Agent/Team').head(21)

Final_Agent/Team
Frank Montro               72
Non Member                 62
QianKun Chen               54
Nadine Ferrata             42
Andres Olaez               37
Salvador Gonzalez          37
Anthony Disano             36
William Volpe              34
Naja Morris                34
Carlos Sanchez             34
Maria Castillo             32
Pablo Galarza              31
Christopher Lobrillo       31
Lynda Sanchez-Werner       30
The Laricy Team            29
Gary Weglarz               28
Grigory Pekarsky           28
Robert Sullivan            26
Gaspar Flores              26
Lorena Ramirez-Carrillo    26
Jose Arroyo                26
Name: count, dtype: int64